The prediction made in the previous section take into account uncertainties due to the fact that a Gaussian process
is stochastic but it doesn’t take into account any uncertainties in the values of the hyperparameters. 
This won’t matter if the hyperparameters are very well constrained by the data but in this case, 
many of the parameters are actually poorly constrained. To take this effect into account, 
we can apply prior probability functions to the hyperparameters and marginalize using Markov chain Monte Carlo (MCMC). 

In [1]:
%pylab inline --no-import-all
import george
from george.kernels import MyDijetKernelSimp


import math
import time
import numpy as np
import scipy.stats as ss
import scipy.special as ssp
import scipy.optimize as op
from iminuit import Minuit

#import ROOT as r
#import warnings
#from rootpy.plotting import Hist, Hist2D, Legend, Canvas
#import rootpy.plotting.root2matplotlib as rplt

Populating the interactive namespace from numpy and matplotlib


In [2]:
xvalO = np.load("../GP_Paper/xvalO.npy")
yvalO = np.load("../GP_Paper/yvalO.npy")
xerrO = np.load("../GP_Paper/xerrO.npy")
xlowO = np.load("../GP_Paper/xlowO.npy")
xval = np.load("../GP_Paper/xval.npy")
yval = np.load("../GP_Paper/yval.npy")
xerr = np.load("../GP_Paper/xerr.npy")
smoothGPFit = np.load("../GP_Paper/initialGPfit.npy")


xvalO2D = xvalO.reshape(-1,1)
yvalO2D = yvalO.reshape(-1,1)

In [3]:
# function to display covariance matrices

def plot_cov(X, C, K, stationary=True):
    K = K #+ 1e-8*np.eye(X.shape[0])
    x = X.flatten()
    
    fig = plt.figure(figsize=(14,5))
    ax1 = fig.add_subplot(131)
    m = ax1.imshow(C, cmap="inferno",
                   interpolation='none',
                   extent=(np.min(X), np.max(X), np.max(X), np.min(X)));
    plt.colorbar(m);
    ax1.set_title("Correlation Matrix")
    ax1.set_xlabel("X")
    ax1.set_ylabel("X")

    ax2 = fig.add_subplot(132)
    m = ax2.imshow(np.log(K), cmap="inferno",
                   interpolation='none',
                   extent=(np.min(X), np.max(X), np.max(X), np.min(X)));
    plt.colorbar(m);
    ax2.set_title("Log(Covariance Matrix)")
    ax2.set_xlabel("X")
    ax2.set_ylabel("X")
    
    ax3 = fig.add_subplot(133)
    m = ax3.imshow(K, cmap="inferno",
                   interpolation='none',
                   extent=(np.min(X), np.max(X), np.max(X), np.min(X)));
    plt.colorbar(m);
    ax3.set_title("Covariance Matrix")
    ax3.set_xlabel("X")
    ax3.set_ylabel("X")
    
    fig = plt.figure(figsize=(13,5))
    ax = fig.add_subplot(111)
    if not stationary:
        ax.plot(x, np.diag(K), "k", lw=2, alpha=0.8)
        #ax2.plot(x, 2e6*np.exp(-(x/300)), c='g')
        ax.set_title("The Diagonal of K")
        ax.set_ylabel("k(x,x)")
        ax.set_xscale('log')
        ax.set_yscale('log')
    else:
        ax.plot(x, K[:,0], "k", lw=2, alpha=0.8)
        ax.set_title("K as a function of x - x'")
        ax.set_ylabel("k(x,x')")
    ax.set_xlabel("X")

    '''
    fig = plt.figure(figsize=(14,4))
    ax = fig.add_subplot(111)
    samples = np.random.multivariate_normal(modelMatrix[0], K, 5).T;
    for i in range(samples.shape[1]):
        ax.plot(x, samples[:,i], color=cmap.inferno(i*0.2), lw=2);
    ax.set_title("Samples from GP Prior")
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xlabel("X")
    '''


In [4]:

#background only plots
def makePrettyPlots(xs, ys, bkgs, title, col = 'g', ymax = 2e5):
    f, (ax1, ax2) = plt.subplots(2, sharex=True, figsize=(12,12), gridspec_kw = {'height_ratios':[3, 1]})
    f.suptitle(title, fontsize=30)
    dataPlot = ax1.errorbar(xs, ys, marker='o', ls='None', yerr = np.sqrt(ys), c='black', markersize=10, label="data+signal")
    bkgPlot, =ax1.plot(xs, bkgs, color=col, linewidth=3.0, label="bkg only")
    ax1.legend()
    ax1.set_ylabel('Events', fontsize=20)
    ax1.tick_params(axis='y', labelsize=20)
    ax1.set_yscale('log', nonposy="clip")
    ax1.set_xscale('log')
    ax1.set_xlim([1000, 7500])
    ax1.set_ylim([0.01, ymax])


  
    #bin by bin significance    
    zvals, chi2 = calcSignificance(ys, bkgs)
    print "chi2", chi2
    h1 = Hist(xlowO,markersize=0)
    for x in range(len(zvals)):
        if zvals[x] == np.inf or zvals[x] == -np.inf: h1[x] = 20
        else: h1[x] = zvals[x]  


    
    h1.fillstyle = 'solid'
    if col == 'g':
         h1.fillcolor = 'green'
    elif col == 'b':
         h1.fillcolor = 'blue'
    elif col == 'r':
         h1.fillcolor = 'red'
    else:
        h1.fillcolor = 'black'
    h1.linecolor = 'black'
    h1.linewidth = 1
    h1.Draw("histX0")
    rplt.bar(h1, xerr=None, yerr=None)
    ax2.axhline(0, color='black', lw=1)
    ax2.tick_params(axis='x', labelsize=20)
    ax2.tick_params(axis='y', labelsize=20)
    ax2.set_xlabel(r"$m_{jj}$ [TeV]", fontsize=30)
    ax2.set_ylabel('Significance', fontsize=20)
    ax2.set_xscale('log')
    ax2.set_xlim([1000, 7500])
    ax2.set_xticks([1000, 2000, 3000, 4000, 5000, 6000, 7000])
    ax2.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
    minor_ticks = np.arange(1000, 7500, 100)
    ax2.set_xticks(minor_ticks, minor=True)  
    labels = ["1","2","3","4","5","6","7"]
    ax2.set_xticklabels(labels)
    ax2.set_ylim([-5, 5])
    
    f.subplots_adjust(hspace=0)
    plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)    
    plt.show()

def calcSignificance(Data, Bkg):
    pvals = []
    zvals = []
    chi2 = 0
    for i, nD in enumerate(Data):
        nB = Bkg[i]
        if nD != 0:
            if nB > nD:
                #pval = 1.-r.TMath.Gamma(nD+1.,nB)
                pval = 1.-ssp.gammainc(nD+1.,nB)
            else:
                #pval = pval = r.TMath.Gamma(nD,nB)
                pval = ssp.gammainc(nD,nB)
            prob = 1.-2.*pval
            if prob > -1 and prob < 1:
                #zval = math.sqrt(2.)*r.TMath.ErfInverse(prob)
                zval = math.sqrt(2.)*ssp.erfinv(prob)
            else:
                zval = np.inf
                #print "crap"
            if zval > 100: zval = 20
            if zval < 0: zval = 0
            if (nD < nB): zval = -zval
           # print nD, nB, pval, prob, zval
        else: zval = 0
            
        zvals.append(zval)
        chi2 += ((nD - nB) ** 2 / abs(nB)) #/len(nB)
    return zvals, chi2

In [5]:

def model_3param(t, params, xErr=xerr): 
    p0, p1, p2 = params
    sqrts = 13000.
    return (p0 * ((1.-t/sqrts)**p1) * (t/sqrts)**(p2))*(xErr)  

def simpleLogPoisson(x, par):
    if x < 0: 
        #print "crap"
        return np.inf
    elif (x == 0): return -1.*par
    else:
        #lnpoisson = x*np.log(par)-par-r.TMath.LnGamma(x+1.)
        lnpoisson = x*np.log(par)-par-ssp.gammaln(x+1.)
        return lnpoisson

In [6]:
class logLike_3ff_minuit:
    def __init__(self, x, y, xe):
        self.x = x
        self.y = y
        self.xe = xe
    def __call__(self, p0, p1, p2):
        params = p0, p1, p2
        bkgFunc = model_3param(self.x, params, self.xe)       
        logL = 0
        for ibin in range(len(self.y)):
            data = self.y[ibin]
            bkg = bkgFunc[ibin]
            logL += -simpleLogPoisson(data, bkg)
        try:
            logL
            return logL
        except:
            return np.inf
def fit_3ff_minuit(num,lnprob):
  #  np.random.seed(1234)
    bestval = np.inf
    bestargs = (0., 0., 0.)
    for i in range(num):
        init0 = np.random.random() * 1.
        init1 = np.random.random() * 8.
        init2 = np.random.random() * 6.
        m = Minuit(lnprob, throw_nan = False, pedantic = False, print_level = 0,
                  p0 = init0, p1 = init1, p2 = init2,
                  error_p0 = 1e-2, error_p1 = 1e-1, error_p2 = 1e-1, 
                  limit_p0 = (0, 100.), limit_p1 = (-100., 100.), limit_p2 = (-100., 100.))
        m.migrad()
        if m.fval < bestval:
            bestval = m.fval
            bestargs = m.args 
    # if Print:
    #     print "min LL",bestval
    #     print "best fit vals", bestargs
    return bestval, bestargs

### 3 parameter covariance matrix

### swift

In [ ]:
'''
def logLike_ind_swift(p, yvals, xvals, xerrs):
    if p[0] < 0 or np.any(-20 > p[1:]) or np.any(p > 20):
        return -np.inf
    bkgFunc = model_3param(xvals, p, xerrs)
    return -0.5 * np.sum( (bkgFunc-yvals)*(bkgFunc-yvals) / bkgFunc)
'''



modelMatrix0=np.zeros([xvalO.size, xvalO.size])

for wi, w in enumerate(xvalO):
    print wi
    if wi < 10: 
        print "under"
        wmin = 0
    else: wmin = wi-10
    if wi > xvalO.size-10: 
        print "over"
        wmax = xvalO.size
    else: wmax = wi+30
    print w, wmin, wmax
    xvals = xvalO[wmin:wmax]
    xerrs = xerrO[wmin:wmax]
    yvals = yvalO[wmin:wmax]

    lnProb = logLike_3ff_minuit(xvals,yvals,xerrs)
    bestval, best_fit_new = fit_3ff_minuit(100, lnProb)

    modelMatrix0[wi,wmin:wmax] = model_3param(xvals,best_fit_new,xerrs)


        
covMat0 = np.cov(modelMatrix0.T)
corMat0 = np.corrcoef(modelMatrix0.T)




0
under
1116.5 0 30
1
under
1149.5 0 31
2
under
1183.0 0 32
3
under
1217.0 0 33
4
under
1251.5 0 34
5
under
1287.0 0 35
6
under
1323.0 0 36
7
under
1359.5 0 37
8
under
1397.0 0 38
9
under
1435.0 0 39
10
1473.5 0 40
11
1513.0 1 41
12
1553.0 2 42
13
1593.5 3 43
14
1635.0 4 44
15
1677.0 5 45
16
1719.5 6 46
17
1763.0 7 47
18
1807.5 8 48
19
1852.5 9 49
20
1898.0 10 50
21
1944.5 11 51
22
1992.0 12 52
23
2040.5 13 53
24
2089.5 14 54
25
2139.0 15 55
26
2189.5 16 56
27
2241.0 17 57
28
2293.5 18 58
29
2347.0 19 59
30
2401.5 20 60
31
2457.0 21 61
32
2513.5 22 62
33
2571.0 23 63
34
2629.5 24 64
35
2689.0 25 65
36
2749.5 26 66
37
2811.0 27 67
38
2873.5 28 68
39
2937.0 29 69
40
3001.5 30 70
41
3067.0 31 71
42
3133.5 32 72
43
3201.0 33 73
44
3270.0 34 74
45
3340.5 35 75
46
3412.0 36 76
47
3484.5 37 77
48
3558.5 38 78
49
3634.0 39 79
50
3710.5 40 80
51
3788.0 41 81
52
3867.0 42 82
53
3947.5 43 83
54
4029.0 44 84
55
4112.0 45 85
56
4196.5 46 86
57
4282.5 47 87
58
4370.0 48 88
59
4459.0 49 89
60
4549.5 

In [ ]:
#plot_cov(np.arange(0,len(xvalO),1).reshape(-1,1), corMat0, covMat0, False)
#makePrettyPlots_bkgFits(xvalO, xvalO, yvalO, (modelMatrix0).tolist(), "", ymax = 2e5)
plt.figure()
m = plt.imshow(covMat0[50:][50:], cmap="inferno",
                   interpolation='none',
                   extent=(1, 10, 10, 1))
plt.colorbar(m)

### smearing width
nx = np.append(xvalO_low, xvalO)
nxe = np.append(xerrO_low, xerrO)
nxl = np.append(xlowO_low, xlowO)

In [ ]:
####run for many shifts and widths of guassian
zs=np.arange(nxl[0], nxl[-1], 1)
steps = 10.*np.random.randn(50)+100.
xs=nx
modelMatrixi=np.zeros([len(steps), xs.size])
for bi, b in enumerate(steps):
    print bi, b
    newycounts=np.zeros(len(xs))
    for xi, x in enumerate(xs):
        for zi, z in enumerate(zs):
            zer= 1.
            pz=model_3param(z,smoothPs,zer)
            gaus = ss.norm.pdf(x, z, b)
            newycounts[xi]+=pz*gaus*nxe[xi] 
    modelMatrixi[bi,:]= newycounts
    #print modelMatrix



In [ ]:
print modelMatrixi[:,11:]
h2=modelMatrixi[:,11:].T
covMati = np.cov(h2)
corMati = np.corrcoef(h2)

plot_cov(nx[11:].reshape(-1,1), corMati, covMati, False)

In [ ]:
makePrettyPlots_bkgFits(nx, xvalO, yvalO, (modelMatrix).tolist(), "", ymax = 2e5)

### smearing width uniform binning

In [ ]:
zs=np.arange(nxl[0], nxl[-1], 1)
#zs=np.arange(xlowO[0], xlowO[-1], 1)
steps = 10.*np.random.randn(50)+100.
xs = np.arange(xlowO[0], xlowO[-1],10)
#xs=xvalO
#xs=nx
modelMatrix5=np.zeros([len(steps), xs.size])
for bi, b in enumerate(steps):
    print bi, b
    newycounts=np.zeros(len(xs))
    for xi, x in enumerate(xs):
        for zi, z in enumerate(zs):
            zer= 1.
            pz=model_3param(z,smoothPs,zer)
            gaus = ss.norm.pdf(x, z, b)
            newycounts[xi]+=pz*gaus*10.
    modelMatrix5[bi,:]= newycounts 

covMat5 = np.cov(modelMatrix5[:,11:])
corMat5 = np.corrcoef(modelMatrix5[:,11:])

plot_cov(nx[11:].reshape(-1,1), corMat5, covMat5, False)

### smearing mean

In [ ]:
nx = np.append(xvalO_low, xvalO)
nxe = np.append(xerrO_low, xerrO)
nxl = np.append(xlowO_low, xlowO)

zs=np.arange(nxl[0], nxl[-1], 1)
#zs=np.arange(xlowO[0], xlowO[-1], 1)
steps = 10.*np.random.randn(50)
#xs = np.arange(xlowO[0], xlowO[-1],10)
#xs=xvalO
xs=nx
modelMatrix3=np.zeros([len(steps), xs.size])
for bi, b in enumerate(steps):
    print bi, b
    newycounts=np.zeros(len(xs))
    for xi, x in enumerate(xs):
        for zi, z in enumerate(zs):
            zer= 1.
            pz=model_3param(z,smoothPs,zer)
            gaus = ss.norm.pdf(x, z+b, 100.)
            newycounts[xi]+=pz*gaus*nxe[xi] #xerrO[xi] #10.
    modelMatrix3[bi,:]= newycounts  

In [ ]:
covMat3 = np.cov(modelMatrix3[:,11:].T)
corMat3 = np.corrcoef(modelMatrix3[:,11:].T)
plot_cov(nx[11:].reshape(-1,1), corMat3, covMat3, False)
makePrettyPlots_bkgFits(nx, xvalO, yvalO, (modelMatrix3).tolist(), "", ymax = 2e5)

### other smearing mean larger

In [ ]:
zs=np.arange(nxl[0], nxl[-1], 1)
steps = 100.*np.random.randn(50)
xs=nx
modelMatrix4=np.zeros([len(steps), xs.size])
for bi, b in enumerate(steps):
    print bi, b
    newycounts=np.zeros(len(xs))
    for xi, x in enumerate(xs):
        for zi, z in enumerate(zs):
            zer= 1.
            pz=model_3param(z,smoothPs,zer)
            gaus = ss.norm.pdf(x, z+b, 100.)
            newycounts[xi]+=pz*gaus*nxe[xi] 
    modelMatrix4[bi,:]= newycounts 
covMat4 = np.cov(modelMatrix4[:,11:-1].T)
corMat4 = np.corrcoef(modelMatrix4[:,11:-1].T)
plot_cov(nx[11:-1].reshape(-1,1), corMat4, covMat4, False)

In [ ]:

mm = np.append(modelMatrix[:,11:], modelMatrix3[:,11:], axis=0)
covMatComb = np.cov(mm.T)
corMatComb = np.corrcoef(mm.T)
plot_cov(nx[11:].reshape(-1,1), corMatComb, covMatComb, False)
makePrettyPlots_bkgFits(nx[11:], xvalO, yvalO, (mm).tolist(), "", ymax = 2e5)

### PDF

In [ ]:
covMat = open("../Downloads/covariancePDF", "r")
bins = open("../Downloads/bins", "r")
binlines = bins.readlines()
newbins=[]
for ib, bl in enumerate(binlines):
    nwl=bl.split(" ")
    newbins.append(float(nwl[1]))
    #newbins.append(float(nwl[2]))
newbins.sort()    
print newbins
covMatPDF= np.zeros((len(newbins),len(newbins)))
corMatPDF= np.zeros((len(newbins),len(newbins)))
lines = covMat.readlines()
for il, line in enumerate(lines):
    line = [float(l) for l in line.split(" ")]
    #print line
    covMatPDF[il,:]=line

for i in range(len(newbins)):
    for j in range(len(newbins)):
        corMatPDF[i][j]=covMatPDF[i][j]/np.sqrt(covMatPDF[i][i]*covMatPDF[j][j])



In [ ]:
plot_cov(np.array(newbins).reshape(-1,1), corMatPDF, covMatPDF, False)

In [ ]:
plt.plot(xvalO,corMat3[5,:], label="JES",c='b')
plt.plot(xvalO,corMati[5,:], label="resolution", c='g')
plt.plot(newbins,corMatPDF[5,:], label="PDF", c= 'r')

plt.figure()
plt.plot(xvalO,corMat3[40,:],label="JES",c='b')
plt.plot(xvalO,corMati[40,:], label="resolution", c='g')
plt.plot(newbins,corMatPDF[10,:],label="PDF", c= 'r')

plt.figure()
plt.plot(xvalO,corMat3[75,:],label="JES",c='b')
plt.plot(xvalO,corMati[75,:], label="resolution", c='g')
plt.plot(newbins,corMatPDF[20,:], label="PDF", c= 'r')

plt.figure()
plt.plot(xvalO,covMat3[5,:],label="JES",c='b')
plt.plot(xvalO,covMati[5,:], label="resolution", c='g')
plt.plot(newbins,covMatPDF[5,:], label="PDF", c= 'r')
plt.yscale('log')

plt.figure()
plt.plot(xvalO,covMat3[40,:],label="JES",c='b')
plt.plot(xvalO,covMati[40,:], label="resolution", c='g')
plt.plot(newbins,covMatPDF[10,:], label="PDF", c= 'r')
plt.yscale('log')
plt.figure()
plt.plot(xvalO,covMat3[75,:],label="JES",c='b')
plt.plot(xvalO,covMati[75,:], label="resolution", c='g')
plt.plot(newbins,covMatPDF[20,:], label="PDF", c= 'r')
plt.yscale('log')
#for i in range(len(corMat[1])):
#    plt.plot(np.arange(len(corMat[1])),(corMat[i,:]))